<a href="https://colab.research.google.com/github/dat-rohit/multimodal-model-Food101/blob/main/Roberta_on_Food101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install datasets
%pip install transformers
! pip install -U accelerate
! pip install -U transformers

In [2]:
from datasets import load_dataset
print('test')

test


In [3]:
food = load_dataset("food101", split="train[:5000]")

In [4]:
print(food)

Dataset({
    features: ['image', 'label'],
    num_rows: 5000
})


In [5]:
food = food.train_test_split(test_size=0.2)

In [6]:
print(food["train"].features["label"])

ClassLabel(names=['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'mac

In [7]:
print(food["train"].features["label"].names)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

from transformers import AutoTokenizer, RobertaModel

import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = RobertaModel.from_pretrained("roberta-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [8]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import accelerate

print(accelerate.__version__)

# Step 1: Load the Food101 dataset
food101_dataset = load_dataset("food101", split="train[:5000]")
food101_dataset = food101_dataset.train_test_split(test_size=0.2)

# Step 2: Prepare the labels
#create dictionnary for labels

labels_train = food["train"].features["label"].names
labels_test = food["test"].features["label"].names

label2id, id2label = dict(), dict()
for i, label in enumerate(labels_train):
    label2id[label] = str(i)
    id2label[str(i)] = label

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(food["train"].features["label"].names))



# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_labels(batch):
    print( batch['label'])
    print(id2label)
    words = [id2label[str(k)] for k in batch['label']]
    print(words)
    return tokenizer(words, padding=True, truncation=True, return_tensors="pt")

# Apply the tokenization to the training and testing sets
tokenized_food_train = food["train"].map(tokenize_labels, batched=True)
tokenized_food_test = food["test"].map(tokenize_labels, batched=True)
# Tokenize the labels
print(tokenized_food_train)
# Step 4: Prepare for training
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_food_train,
    eval_dataset=tokenized_food_test
)

trainer.train()


0.24.1


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

[77, 79, 6, 10, 53, 20, 77, 20, 79, 79, 79, 79, 10, 6, 79, 6, 10, 53, 79, 10, 79, 79, 77, 77, 6, 20, 6, 79, 53, 81, 53, 81, 79, 81, 81, 10, 81, 79, 81, 6, 10, 6, 79, 6, 20, 6, 79, 53, 79, 6, 10, 81, 10, 81, 81, 81, 79, 6, 20, 6, 77, 20, 81, 53, 10, 6, 79, 79, 10, 53, 6, 6, 81, 53, 81, 6, 6, 10, 53, 81, 20, 53, 77, 53, 81, 81, 20, 79, 81, 20, 77, 20, 10, 6, 81, 79, 53, 53, 10, 20, 77, 81, 81, 81, 20, 10, 79, 77, 20, 77, 81, 20, 79, 79, 79, 6, 10, 81, 77, 81, 10, 79, 6, 10, 20, 79, 53, 77, 81, 81, 79, 81, 10, 53, 81, 10, 79, 53, 81, 79, 10, 6, 20, 6, 10, 79, 20, 81, 10, 53, 10, 6, 20, 10, 79, 53, 53, 20, 20, 79, 53, 10, 53, 10, 53, 10, 20, 77, 77, 10, 10, 79, 53, 77, 10, 20, 20, 20, 20, 79, 20, 20, 77, 79, 10, 79, 53, 20, 81, 20, 77, 53, 53, 81, 79, 10, 81, 10, 81, 77, 20, 79, 77, 77, 81, 53, 77, 20, 10, 20, 53, 81, 53, 10, 79, 53, 79, 6, 79, 20, 77, 81, 6, 10, 53, 10, 77, 6, 79, 81, 20, 79, 20, 53, 81, 79, 77, 53, 20, 79, 79, 77, 79, 53, 6, 79, 20, 20, 6, 10, 20, 81, 6, 10, 6, 79, 10, 7

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[79, 10, 79, 79, 77, 20, 79, 6, 53, 6, 10, 77, 77, 77, 6, 10, 77, 10, 81, 77, 10, 79, 10, 53, 6, 6, 20, 79, 77, 77, 6, 6, 10, 79, 53, 20, 20, 53, 77, 77, 20, 77, 79, 77, 79, 53, 6, 53, 53, 77, 20, 10, 79, 53, 77, 79, 79, 81, 10, 20, 79, 20, 20, 53, 20, 6, 6, 81, 6, 53, 10, 20, 6, 79, 10, 20, 20, 53, 20, 77, 10, 10, 81, 53, 20, 53, 10, 81, 20, 77, 79, 53, 20, 20, 81, 6, 6, 77, 10, 77, 77, 10, 53, 81, 10, 20, 10, 20, 6, 81, 81, 79, 81, 79, 53, 79, 20, 77, 77, 20, 6, 79, 20, 77, 6, 81, 53, 79, 20, 81, 77, 81, 79, 10, 81, 53, 53, 10, 77, 6, 10, 79, 10, 81, 77, 10, 6, 6, 20, 53, 6, 20, 6, 20, 6, 77, 53, 77, 81, 10, 6, 81, 53, 79, 20, 20, 77, 81, 6, 6, 79, 6, 81, 53, 6, 79, 6, 10, 79, 79, 79, 10, 81, 53, 77, 79, 10, 77, 20, 20, 53, 10, 81, 77, 6, 81, 79, 20, 20, 53, 53, 53, 10, 53, 79, 10, 20, 53, 53, 20, 81, 53, 6, 77, 6, 20, 79, 20, 20, 53, 10, 53, 20, 20, 79, 81, 53, 53, 53, 81, 6, 81, 53, 81, 6, 53, 6, 53, 81, 53, 79, 81, 79, 81, 6, 53, 79, 79, 79, 20, 20, 6, 79, 20, 79, 79, 20, 20, 6, 1

Step,Training Loss
500,0.834300


TrainOutput(global_step=750, training_loss=0.5568582440217336, metrics={'train_runtime': 2272.4411, 'train_samples_per_second': 5.281, 'train_steps_per_second': 0.33, 'total_flos': 43205027544000.0, 'train_loss': 0.5568582440217336, 'epoch': 3.0})